In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df=pd.read_csv('Patient_Data.csv')

In [3]:
df.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


In [4]:
df.shape

(6, 7)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [6]:
df['CheckInTime']=df['CheckInTime'].astype('datetime64[ns]')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PatientID       6 non-null      int64         
 1   Name            6 non-null      object        
 2   Department      6 non-null      object        
 3   Doctor          6 non-null      object        
 4   BillAmount      4 non-null      float64       
 5   ReceptionistID  6 non-null      int64         
 6   CheckInTime     6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 468.0+ bytes


In [8]:
df.drop(columns=['ReceptionistID','CheckInTime','Name'],inplace=True)

In [9]:
df.shape

(6, 4)

In [10]:
df.columns

Index(['PatientID', 'Department', 'Doctor', 'BillAmount'], dtype='object')

In [11]:
df.isnull().sum()

PatientID     0
Department    0
Doctor        0
BillAmount    2
dtype: int64

In [12]:
# filling the missing values in bill amount 
df['BillAmount']=df['BillAmount'].fillna(df['BillAmount'].mean())

In [13]:
df['BillAmount']=df['BillAmount'].astype(int)

In [14]:
# groupby to find total bill amount per department.
df.groupby('Department')['BillAmount'].sum()

Department
Cardiology     16200
Dermatology     5925
Neurology       5925
Orthopedics     7500
Name: BillAmount, dtype: int32

In [15]:
# Remove duplicate patient records based on PatientID
new_df=df.drop_duplicates()

In [16]:
new_df

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000
1,102,Neurology,Dr. John,5925
2,103,Orthopedics,Dr. Lee,7500
3,104,Cardiology,Dr. Smith,6200
4,105,Dermatology,Dr. Rose,5925


In [17]:
df1=pd.read_csv('Billing_Data.csv')

In [18]:
df1.head()

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [21]:
#Merge the billing dataset with patient dataset on PatientID.
merged_df=pd.merge(df1,new_df,on='PatientID',how='outer')
merged_df

,PatientID,InsuranceCovered,FinalAmount,Department,Doctor,BillAmount
0,101,2000,3000,Cardiology,Dr. Smith,5000
1,102,1500,3500,Neurology,Dr. John,5925
2,103,2500,5000,Orthopedics,Dr. Lee,7500
3,104,3000,3200,Cardiology,Dr. Smith,6200
4,105,1000,4000,Dermatology,Dr. Rose,5925


In [23]:
merged_df.columns

Index(['PatientID', 'InsuranceCovered', 'FinalAmount', 'Department', 'Doctor',
       'BillAmount'],
      dtype='object')

In [33]:
#Concatenate an additional DataFrame that contains new patients for the current week (row-wise).
df_new = pd.DataFrame({
    'PatientID': [4, 5],
    'InsuranceCovered': [400, 700],
    'FinalAmount': [900, 1700],
    'Department': ['Radiology', 'Cardiology'],
    'Doctor': ['Dr. Daniel', 'Dr. Era'],
    'BillAmount': [1300, 2000]
})

In [34]:
updated_df=pd.concat([merged_df,df_new],axis=0,ignore_index=True)
updated_df

,PatientID,InsuranceCovered,FinalAmount,Department,Doctor,BillAmount
0,101,2000,3000,Cardiology,Dr. Smith,5000
1,102,1500,3500,Neurology,Dr. John,5925
2,103,2500,5000,Orthopedics,Dr. Lee,7500
3,104,3000,3200,Cardiology,Dr. Smith,6200
4,105,1000,4000,Dermatology,Dr. Rose,5925
5,4,400,900,Radiology,Dr. Daniel,1300
6,5,700,1700,Cardiology,Dr. Era,2000


In [37]:
df_new_cats = pd.DataFrame({
    'PaymentMode': ['Cash', 'Credit Card', 'UPI', 'Insurance', 'Debit Card','Cash','UPI',],
    'Feedback': ['Excellent', 'Good', 'Average', 'Poor', 'Good','Poor','Good']
})

In [38]:
#Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).
pd.concat([updated_df,df_new_cats],axis=1)

,PatientID,InsuranceCovered,FinalAmount,Department,Doctor,BillAmount,PaymentMode,Feedback
0,101,2000,3000,Cardiology,Dr. Smith,5000,Cash,Excellent
1,102,1500,3500,Neurology,Dr. John,5925,Credit Card,Good
2,103,2500,5000,Orthopedics,Dr. Lee,7500,UPI,Average
3,104,3000,3200,Cardiology,Dr. Smith,6200,Insurance,Poor
4,105,1000,4000,Dermatology,Dr. Rose,5925,Debit Card,Good
5,4,400,900,Radiology,Dr. Daniel,1300,Cash,Poor
6,5,700,1700,Cardiology,Dr. Era,2000,UPI,Good
